## 1. Implemeting very simple tokenizer

## Importing libraries and checking versions

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0
tiktoken version: 0.9.0


## Importing War and Peace as a txt file

In [75]:
import os
import urllib.request
import ssl
import certifi

# Use certifi to set the correct CA bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())

file_path = "war_and_peace.txt"
if not os.path.exists(file_path):
    url = "https://www.gutenberg.org/cache/epub/2600/pg2600.txt"
    with urllib.request.urlopen(url, context=ssl_context) as response, open(file_path, 'wb') as out_file:
        out_file.write(response.read())
    print("Download complete.")
else:
    print("File already exists.")

Download complete.


## Removing Gutenberg headers and footers to extract raw text

In [76]:
file_path = "../preprocessing War & Peace/war_and_peace.txt"

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK WAR AND PEACE ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK WAR AND PEACE ***"

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Find the content between the start and end markers
inside_book = False
cleaned_lines = []

for line in lines:
    if start_marker in line:
        inside_book = True
        continue  # Skip the marker line itself
    elif end_marker in line:
        break  # Stop when we reach the end marker
    if inside_book:
        cleaned_lines.append(line)

# Remove Table of Contents and any CHAPTER or BOOK headings
final_lines = []
in_toc = True

for line in cleaned_lines:
    stripped = line.strip()

    # Stop skipping lines after the TOC ends and real content begins
    if in_toc and stripped.startswith("BOOK ONE"):
        in_toc = False

    # Skip lines during Table of Contents
    if in_toc:
        continue

    # Skip chapter/section headings
    if re.match(r"^(BOOK\s+[A-Z]+:|\s*CHAPTER\s+[IVXLCDM]+|FIRST EPILOGUE|SECOND EPILOGUE)", stripped, re.IGNORECASE):
        continue

    final_lines.append(line)

# Remove extra blank lines at start and end
while final_lines and final_lines[0].strip() == "":
    final_lines.pop(0)
while final_lines and final_lines[-1].strip() == "":
    final_lines.pop()

# Save the cleaned file
with open(file_path, "w", encoding="utf-8") as f:
    f.writelines(final_lines)

print("Cleaned: Removed Table of Contents, headings, and retained only the main story.")

Cleaned: Removed Table of Contents, headings, and retained only the main story.


## Tokenization

In [77]:
with open("../preprocessing War & Peace/war_and_peace.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 3197019
“Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, i


In [78]:
import re

preprocessed = re.split(r'([,.:;?_—*!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:3500])

['“Well', ',', 'Prince', ',', 'so', 'Genoa', 'and', 'Lucca', 'are', 'now', 'just', 'family', 'estates', 'of', 'the', 'Buonapartes', '.', 'But', 'I', 'warn', 'you', ',', 'if', 'you', 'don’t', 'tell', 'me', 'that', 'this', 'means', 'war', ',', 'if', 'you', 'still', 'try', 'to', 'defend', 'the', 'infamies', 'and', 'horrors', 'perpetrated', 'by', 'that', 'Antichrist', '—', 'I', 'really', 'believe', 'he', 'is', 'Antichrist', '—', 'I', 'will', 'have', 'nothing', 'more', 'to', 'do', 'with', 'you', 'and', 'you', 'are', 'no', 'longer', 'my', 'friend', ',', 'no', 'longer', 'my', '‘faithful', 'slave', ',', '’', 'as', 'you', 'call', 'yourself', '!', 'But', 'how', 'do', 'you', 'do', '?', 'I', 'see', 'I', 'have', 'frightened', 'you', '—', 'sit', 'down', 'and', 'tell', 'me', 'all', 'the', 'news', '.', '”', 'It', 'was', 'in', 'July', ',', '1805', ',', 'and', 'the', 'speaker', 'was', 'the', 'well-known', 'Anna', 'Pávlovna', 'Schérer', ',', 'maid', 'of', 'honor', 'and', 'favorite', 'of', 'the', 'Empress

In [79]:
print(len(preprocessed))

655650


## Converting tokens into token IDs by creating a vocabulary

In [80]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)
print(all_words)

21316
['!', '(', ')', '*', ',', '-', '--', '.', '000', '1', '10', '100', '102', '11', '110', '120', '13', '130', '13th', '140', '15', '15/4', '150', '15y', '160', '17', '178', '1789', '1797', '17th', '18', '1805', '1805-6', '1806', '1807', '1808', '1809', '1810', '1811', '1812', '1813', '1815', '1820', '18th', '1st', '2', '20', '217', '22', '22nd', '23', '23rd', '24', '24th', '25', '25th', '26', '27', '27th', '3', '30', '31', '3rd', '4', '40', '4th', '4x', '5', '50', '6', '6-me', '60', '62', '666', '671', '6th', '7', '70', '700', '7th', '8', '80', '86th', '8th', '9', '90', '9th', ':', ';', '=', '?', 'A', 'ABC', 'Abandoning', 'Abashed', 'Abbé', 'About', 'Above', 'Abroad', 'Abrámovna', 'Absolute', 'Academy', 'According', 'Achilles', 'Achilles’', 'Achtung', 'Across', 'Acts', 'Adam', 'Adieu', 'Adjutant', 'Adjutant-General', 'Adjutants', 'Admit', 'Admitting', 'Adonai', 'Adorable', 'Adráksin', 'Advisers', 'Affairs', 'Afraid', 'Africa', 'Africans', 'After', 'Afterwards', 'Again', 'Agitated', 

In [81]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 100:
        break

('!', 0)
('(', 1)
(')', 2)
('*', 3)
(',', 4)
('-', 5)
('--', 6)
('.', 7)
('000', 8)
('1', 9)
('10', 10)
('100', 11)
('102', 12)
('11', 13)
('110', 14)
('120', 15)
('13', 16)
('130', 17)
('13th', 18)
('140', 19)
('15', 20)
('15/4', 21)
('150', 22)
('15y', 23)
('160', 24)
('17', 25)
('178', 26)
('1789', 27)
('1797', 28)
('17th', 29)
('18', 30)
('1805', 31)
('1805-6', 32)
('1806', 33)
('1807', 34)
('1808', 35)
('1809', 36)
('1810', 37)
('1811', 38)
('1812', 39)
('1813', 40)
('1815', 41)
('1820', 42)
('18th', 43)
('1st', 44)
('2', 45)
('20', 46)
('217', 47)
('22', 48)
('22nd', 49)
('23', 50)
('23rd', 51)
('24', 52)
('24th', 53)
('25', 54)
('25th', 55)
('26', 56)
('27', 57)
('27th', 58)
('3', 59)
('30', 60)
('31', 61)
('3rd', 62)
('4', 63)
('40', 64)
('4th', 65)
('4x', 66)
('5', 67)
('50', 68)
('6', 69)
('6-me', 70)
('60', 71)
('62', 72)
('666', 73)
('671', 74)
('6th', 75)
('7', 76)
('70', 77)
('700', 78)
('7th', 79)
('8', 80)
('80', 81)
('86th', 82)
('8th', 83)
('9', 84)
('90', 85)
('9th',

In [82]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_—*!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"—*()\'])', r'\1', text)
        return text

Encoding words into tokens

In [83]:
tokenizer = SimpleTokenizerV1(vocab)
# Text from the book
text = """The militiamen with stretchers who were called up stood behind the officers."""
ids = tokenizer.encode(text)
print(ids)

[2769, 12408, 19707, 17399, 19617, 19535, 5142, 18983, 17308, 4469, 17977, 13161, 7]


Decoding the IDs back into text

In [84]:
tokenizer.decode(ids)

'The militiamen with stretchers who were called up stood behind the officers.'

## Unknown token encoding

In [85]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [86]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_—*!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"—*()\'])', r'\1', text)
        return text

In [87]:
tokenizer = SimpleTokenizerV2(vocab)
# Check if War and Peace contains my name in the vocabulary
text1 = "Hello, do you like tea?"
text2 = "Hello, my name is Kamilla."

text = " <|endoftext|> ".join((text1, text2))

print(text)

tokenizer.encode(text)
tokenizer.decode(tokenizer.encode(text))

Hello, do you like tea? <|endoftext|> Hello, my name is Kamilla.


'<|unk|>, do you like tea? <|endoftext|> <|unk|>, my name is <|unk|>.'

## 2. Byte-pair encoding

In [88]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

tiktoken version: 0.9.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [89]:
with open("../preprocessing War & Peace/war_and_peace.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
enc_sample = enc_text[50:]

844916


In [100]:
context_size = 100
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

x: [4404, 262, 1167, 321, 444, 290, 26611, 35524, 416, 326, 198, 13217, 488, 1585, 960, 40, 1107, 1975, 339, 318, 3738, 488, 1585, 960, 40, 481, 423, 2147, 198, 3549, 284, 466, 351, 345, 290, 345, 389, 645, 2392, 616, 1545, 11, 645, 2392, 616, 198, 447, 246, 41751, 913, 11778, 11, 447, 247, 355, 345, 869, 3511, 0, 887, 703, 466, 345, 466, 30, 314, 766, 314, 198, 14150, 24776, 345, 960, 48937, 866, 290, 1560, 502, 477, 262, 1705, 13, 447, 251, 198, 198, 1026, 373, 287, 2901, 11, 1248, 2713, 11, 290, 262, 10834, 373, 262, 880]
y:      [262, 1167, 321, 444, 290, 26611, 35524, 416, 326, 198, 13217, 488, 1585, 960, 40, 1107, 1975, 339, 318, 3738, 488, 1585, 960, 40, 481, 423, 2147, 198, 3549, 284, 466, 351, 345, 290, 345, 389, 645, 2392, 616, 1545, 11, 645, 2392, 616, 198, 447, 246, 41751, 913, 11778, 11, 447, 247, 355, 345, 869, 3511, 0, 887, 703, 466, 345, 466, 30, 314, 766, 314, 198, 14150, 24776, 345, 960, 48937, 866, 290, 1560, 502, 477, 262, 1705, 13, 447, 251, 198, 198, 1026, 373, 28

In [93]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 defend ---->  the
 defend the ---->  inf
 defend the inf ----> am
 defend the infam ----> ies
 defend the infamies ---->  and
 defend the infamies and ---->  horrors
 defend the infamies and horrors ---->  perpetrated
 defend the infamies and horrors perpetrated ---->  by
 defend the infamies and horrors perpetrated by ---->  that
 defend the infamies and horrors perpetrated by that ----> 

 defend the infamies and horrors perpetrated by that
 ----> Ant
 defend the infamies and horrors perpetrated by that
Ant ----> ich
 defend the infamies and horrors perpetrated by that
Antich ----> rist
 defend the infamies and horrors perpetrated by that
Antichrist ----> —
 defend the infamies and horrors perpetrated by that
Antichrist— ----> I
 defend the infamies and horrors perpetrated by that
Antichrist—I ---->  really
 defend the infamies and horrors perpetrated by that
Antichrist—I really ---->  believe
 defend the infamies and horrors perpetrated by that
Antichrist—I really believe ---->  he

In [94]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0


In [96]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [97]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [98]:
with open("../preprocessing War & Peace/war_and_peace.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [99]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[ 447,  250, 5779,   11]]), tensor([[ 250, 5779,   11, 9005]])]


In [101]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 250, 5779,   11, 9005]]), tensor([[5779,   11, 9005,   11]])]


## 3. Embedding layer